In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../experiments/')
import os
import scipy
import pandas as pd
import joblib
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from dict_hash import sha256
import numpy as np
from torch.autograd import grad
import torch
from tqdm import tqdm
from torch.autograd.functional import jacobian
from torch.func import jacfwd
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
import torchhd
from data import BabyLMDataset, BABYLM_ROOT_DIR
from lm import HDLM

In [2]:
checkpoint = 'gpt2'
tok = AutoTokenizer.from_pretrained(checkpoint)
start = 'Roses are red, violets are'
end = ' blue'
lm = HDLM(checkpoint, device='cpu', emb_size=10000, learning_rate=0.1)

print('tokens as strings', lm.tokenizer.tokenize(start))
input_ids = lm.tokenizer.encode(start, return_tensors='pt').to(lm.device)
print('tokens as ids', input_ids)

lm.get_embs(input_ids).shape

next_emb = lm.next_emb_from_token_ids(input_ids)
print('next_emb', next_emb.shape)

next_token_id = lm.emb_to_token_id(next_emb)
print('next_token', next_token_id, lm.tokenizer.decode(next_token_id))

# update vocab emb
next_token_correct_id = lm.tokenizer(end)['input_ids']
lm.update_vocab_emb(next_emb, next_token_correct_id)


next_token_id = lm.emb_to_token_id(next_emb)
print('next_token', next_token_id, lm.tokenizer.decode(next_token_id))
assert lm.tokenizer.decode(next_token_id) == end

tokens as strings ['R', 'oses', 'Ġare', 'Ġred', ',', 'Ġv', 'io', 'lets', 'Ġare']
tokens as ids tensor([[  49, 4629,  389, 2266,   11,  410,  952, 5289,  389]])
next_emb torch.Size([10000])
next_token tensor(47514) Bah
next_token tensor(4171)  blue


# Training loop

In [11]:
checkpoint = 'gpt2'
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
start = 'Roses are red, violets are'
end = ' blue'
lm = HDLM(checkpoint, device='cpu', emb_size=10000, learning_rate=0.1)
dset = BabyLMDataset(join(BABYLM_ROOT_DIR, 'babylm_dev', 'full.joblib'), 10)
dset_test = BabyLMDataset(
    join(BABYLM_ROOT_DIR, 'babylm_test', 'full.joblib'), 10)

In [12]:
perplexity = lm.calc_perplexity(dset, n_examples=100, train=False)
perplexity_test = lm.calc_perplexity(dset_test, n_examples=100, train=False)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:46<00:00,  2.14it/s]


In [13]:
print('perp', perplexity, 'perp test', perplexity_test)

perp 10.765946388244629 perp test 10.876726150512695


In [14]:
lm.calc_perplexity(dset, n_examples=100, train=True)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:22<00:00,  4.36it/s]


17.713855743408203

In [15]:
perplexity = lm.calc_perplexity(dset, n_examples=100, train=False)
perplexity_test = lm.calc_perplexity(dset_test, n_examples=100, train=False)
print('perp', perplexity, 'perp test', perplexity_test)

100%|██████████| 100/100 [00:48<00:00,  2.06it/s]

perp 21.905921936035156 perp test 39.63344955444336


In [33]:
# run an example
toks, next = dset[0]
print(repr(lm.tokenizer.decode(toks)), '->', repr(lm.tokenizer.decode(next)))
emb_pred = lm.next_emb_from_token_ids(toks)
tok_pred = lm.tokenizer.decode(lm.emb_to_token_id(emb_pred))
print('pred', repr(tok_pred))

'Doilies, Pam? \nHello, Br' -> 'idget'
pred '.'
